In [ ]:
!pip install gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
import os
import requests
from gradio_client import Client

TOKEN = 'YOUR_TOKEN'
client = Client("https://marcosv-instructir.hf.space/--replicas/srty7/")

def get_chat_id(update):
    chat_id = update['message']['chat']['id']
    return chat_id

def send_message(chat_id, text):
    url = f'https://api.telegram.org/bot{TOKEN}/sendMessage'
    params = {'chat_id': chat_id, 'text': text}
    requests.post(url, params=params)

def send_photo(chat_id, photo_path):
    url = f'https://api.telegram.org/bot{TOKEN}/sendPhoto'
    photo_files = {'photo': open(photo_path, 'rb')}
    requests.post(url, params={'chat_id': chat_id}, files=photo_files)

def download_photo(file_id):
    download_url = f'https://api.telegram.org/bot{TOKEN}/getFile?file_id={file_id}'
    response = requests.get(download_url)
    file_path = response.json()['result']['file_path']
    photo_url = f'https://api.telegram.org/file/bot{TOKEN}/{file_path}'
    response = requests.get(photo_url)

    with open('received_photo.jpg', 'wb') as file:
        file.write(response.content)

def handle_message(update):
    chat_id = get_chat_id(update)
    message_text = update['message']['text']

    if message_text == '/start':
        send_message(chat_id, 'Hello, welcome to bot, please send me the picture you want.')
    # elif message_text == '/photo':
    #     send_message(chat_id, 'please send me the picture you want.')

def handle_photo(update):
    chat_id = get_chat_id(update)
    file_id = update['message']['photo'][-1]['file_id']

    download_photo(file_id)
    send_message(chat_id, 'Please enter the desired prompt as "/prompt YOUR PARAMETER"')

def handle_prompt(update):
    chat_id = get_chat_id(update)
    parameter_text = update['message']['text']

    prompt = parameter_text.split(' ', 1)[1]

    result = client.predict(
        "received_photo.jpg",    # filepath  in 'Input' Image component
        prompt,  # str  in 'Prompt' Textbox component
        api_name="/predict"
    )

    # send_message(chat_id, f'result: {result}')
    send_message(chat_id, 'Changes applied! Sending new image...')
    send_photo(chat_id, result)

    os.remove('received_photo.jpg')

def main():
    offset = None

    while True:
        updates_url = f'https://api.telegram.org/bot{TOKEN}/getUpdates'
        params = {'offset': offset, 'timeout': 30}
        response = requests.get(updates_url, params=params)
        updates = response.json()['result']

        for update in updates:
            offset = update['update_id'] + 1

            if 'message' in update:
                if 'text' in update['message']:
                    message_text = update['message']['text']
                    if message_text.startswith('/prompt'):
                        handle_prompt(update)
                    else:
                        handle_message(update)
                elif 'photo' in update['message']:
                    handle_photo(update)

if __name__ == '__main__':
    main()

Loaded as API: https://marcosv-instructir.hf.space/--replicas/srty7/ ✔
